In [2]:
import yt
from matplotlib import rc_context
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import numpy as np
print(yt.__version__)

4.4.0


In [3]:
Norm = np.array([1.0,
  1./3.,
  1./3.,
  1./3.,
  2./3.,
  4./3.,
  4./9.,
  1./9.,
  1./9.,
  1./9.,
  2./3.,
  2./3.,
  2./3.,
  2./9.,
  2./9.,
  2./9.,
  2.0,
  4./3.,
  4./9.])
print(Norm.shape)

(19,)


In [5]:
############    modify if needed    ############################
step1 = 702000
step2 = 900000
plot_int = 2000
temperature = 1e-5
alpha0 = 0.
cs2 = 1./3.
nx = 32
ny = 32
nz = 32
ncomp = 19  # nvel in LBM
###############################################################

Nframes = (step2 - step1)//plot_int + 1
fnoise_ndarray = np.zeros((Nframes, ncomp, nx, ny, nz))
gnoise_ndarray = np.zeros((Nframes, ncomp, nx, ny, nz))

plot_file_fnoise_root = "./data_mixture/lbm_data_shshan_alpha0_" + "{:.2f}".format(alpha0) +\
          "_xi_" + "{:.1e}".format(temperature) + "_size{:d}-{:d}-{:d}_continue/data_fnoise/fn".format(nx,ny,nz)
plot_file_gnoise_root = "./data_mixture/lbm_data_shshan_alpha0_" + "{:.2f}".format(alpha0) +\
          "_xi_" + "{:.1e}".format(temperature) + "_size{:d}-{:d}-{:d}_continue/data_gnoise/gn".format(nx,ny,nz)

'''
plot_file_fnoise = plot_file_fnoise_root + f"{step1:06d}"
ds0 = yt.load(plot_file_fnoise)
print(ds0.field_list)
'''

# load in noise multifab of each frame into ndarray
for n in range(step1, step2+1, plot_int):
    plot_file_fnoise = plot_file_fnoise_root + f"{n:07d}"
    plot_file_gnoise = plot_file_gnoise_root + f"{n:07d}"
    ds0 = yt.load(plot_file_fnoise)
    fnoise_data_frame = ds0.covering_grid(
    level=0, left_edge=ds0.domain_left_edge, dims=ds0.domain_dimensions)
    ds1 = yt.load(plot_file_gnoise)
    gnoise_data_frame = ds1.covering_grid(
    level=0, left_edge=ds1.domain_left_edge, dims=ds1.domain_dimensions)
    for a in range(0, ncomp):
        fnoise_ndarray[(n-step1)//plot_int, a, :,:,:] = fnoise_data_frame["boxlib", "fa{:d}".format(a)]
        gnoise_ndarray[(n-step1)//plot_int, a, :,:,:] = gnoise_data_frame["boxlib", "ga{:d}".format(a)]
    

print(fnoise_ndarray.shape)


yt : [INFO     ] 2025-03-17 15:25:07,690 Parameters: current_time              = 702000.0
yt : [INFO     ] 2025-03-17 15:25:07,691 Parameters: domain_dimensions         = [32 32 32]
yt : [INFO     ] 2025-03-17 15:25:07,693 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2025-03-17 15:25:07,694 Parameters: domain_right_edge         = [1. 1. 1.]
yt : [INFO     ] 2025-03-17 15:25:07,769 Parameters: current_time              = 702000.0
yt : [INFO     ] 2025-03-17 15:25:07,770 Parameters: domain_dimensions         = [32 32 32]
yt : [INFO     ] 2025-03-17 15:25:07,772 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2025-03-17 15:25:07,773 Parameters: domain_right_edge         = [1. 1. 1.]
yt : [INFO     ] 2025-03-17 15:25:08,397 Parameters: current_time              = 704000.0
yt : [INFO     ] 2025-03-17 15:25:08,398 Parameters: domain_dimensions         = [32 32 32]
yt : [INFO     ] 2025-03-17 15:25:08,398 Parameters: domain_left_edge          = [0. 0

(100, 19, 32, 32, 32)


In [11]:
tau = 1.
lbda = 1./tau
lbda_bar = lbda/(1+0.5*lbda)
factor1 = 2.*lbda_bar - lbda_bar**2
factor2 = temperature/cs2*Norm

fnoise_mode1_corv = np.zeros((ncomp, nx, ny, nz))
for a in range(1, 3+1):
    for n in range(Nframes):
        fnoise_mode1_corv[a,:,:,:] = fnoise_mode1_corv[a,:,:,:] + fnoise_ndarray[n,1,:,:,:]**2
    fnoise_mode1_corv[a,:,:,:] = fnoise_mode1_corv[a,:,:,:]/Nframes

fnoise_mode1_corv_normalized = fnoise_mode1_corv/temperature/factor1/0.5

a = 3
print(fnoise_mode1_corv_normalized[a,:,:,:])

print("numerical corvariance divided by theoretical value: mean = {:.5f} and var = {:.5f}"\
       .format(np.mean(fnoise_mode1_corv_normalized[a,:,:,:]), np.var(fnoise_mode1_corv_normalized[a,:,:,:])))


[[[1.10695194 1.12886647 1.04680512 ... 1.04955393 1.21910952 1.05176628]
  [1.06555152 1.19560701 0.95408472 ... 0.99220209 0.60759553 0.99087678]
  [0.75857105 1.00951583 0.98620155 ... 1.0663344  1.14979083 1.26248578]
  ...
  [0.86229449 1.06156359 1.21940465 ... 1.10864654 1.04122136 0.91192364]
  [1.33301617 0.91324156 1.22604243 ... 0.96404413 1.10145693 1.05753495]
  [0.87596562 0.94223011 1.08648575 ... 0.91095532 0.9797473  0.96398179]]

 [[1.0938635  1.26289062 1.02221544 ... 0.8221377  0.87930151 1.0646986 ]
  [0.92267954 0.8885735  1.15457877 ... 1.19991549 0.92505836 0.76847127]
  [1.05214851 0.85155742 0.99128226 ... 1.06222468 0.95023471 0.86063161]
  ...
  [1.02165349 0.91461026 1.17225303 ... 0.93597758 0.99998694 1.14255103]
  [0.87825757 1.01329982 1.11330337 ... 0.82499602 1.19493465 0.74556991]
  [0.86805673 1.27167915 0.89702051 ... 0.8642294  1.0036747  0.81919564]]

 [[0.69994571 1.19194962 1.16911601 ... 1.15374724 0.99815856 0.93461485]
  [1.02279429 1.088662